In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_input_densenet
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_input_mobilenet
from tensorflow.keras.models import load_model
import os
import pandas as pd

In [ ]:
CLASSES = {
    "ACK": "ACK",
    "MEL": "MEL",
    "BCC": "BCC",
    "SCC": "SCC",
    "SEK": "SEK",
    "NEV": "NEV"
}
model = load_model("model.h5")


def load_image(image_path, target_size=(224, 224)):
    """
    Loads an input image into PIL format of specified size
    """
    img = image.load_img(path=image_path, target_size=target_size)
    return img

def preprocess_image(loaded_image, model_name):
    """
    Preprocesses a loaded image based on the specified model
    """
    img_array = image.img_to_array(loaded_image)
    img_batch = np.expand_dims(img_array, axis=0)
    
    if model_name == "resnet":
        processed_img = preprocess_input_resnet50(img_batch)
    elif model_name == "densenet":
        processed_img = preprocess_input_densenet(img_batch)
    elif model_name == "mobilenet":
        processed_img = preprocess_input_mobilenet(img_batch)
    else:
        raise ValueError("Invalid model name")
    
    return processed_img

def show_preprocess_image(loaded_image, model_names):
    """
    Shows loaded image and preprocesses it for the specified models
    """
    # Display image
    plt.figure(figsize=(3, 3), dpi=100)
    plt.imshow(loaded_image)
    
    # Preprocess image
    processed_images = []
    for model_name in model_names:
        processed_img = preprocess_image(loaded_image, model_name)
        processed_images.append(processed_img)
    
    return processed_images

def image_predict(preprocessed_images, non_image_features, model):
    """
    Returns class probabilities for given preprocessed images and non-image features,
    based on the loaded model
    """
    predictions = model.predict([preprocessed_images, non_image_features])
    probabilities = np.round(predictions[0], 6)

    class_probabilities = dict(zip(CLASSES, probabilities))
    
    return class_probabilities

In [ ]:
def predict_for_all_customers(model):
    
    # Load non-image features
    test_df = pd.read_csv("./Augmentation/test_df.csv")
    test_df = test_df.sample(frac=1).reset_index()

    # Initialize an empty list to store the results
    results = []

    for _, row in test_df.iterrows():
        # Extract customer information
        patient_name = row["patient"]
        real_class = row["diagnostic"]

        # Construct the full image path
        test_image_path = os.path.join('./cancer/images/test', row["img_id"])

        # Load and preprocess the user-selected image
        loaded_image = load_image(test_image_path)
        model_names = ["resnet", "densenet", "mobilenet"]
        processed_images = [preprocess_image(loaded_image, model_name) for model_name in model_names]

        # Replicate non_image_features to match the batch size
        non_image_features = np.array(row.drop(["diagnostic", "img_id", "patient", "index", "level_0"]))

        # Convert non-image features to float data type
        non_image_features = non_image_features.astype(float)

        num_samples = processed_images[0].shape[0]
        non_image_features = np.tile(non_image_features, (num_samples, 1))

        # Make predictions on the test image
        predictions = image_predict(processed_images, non_image_features, model)

        # Find the highest-scoring prediction
        max_prob = max(predictions.values())
        predicted_class = [k for k, v in predictions.items() if v == max_prob][0]

        # Get the full name of the predicted class
        predicted_class_name = CLASSES.get(predicted_class)

        # Append the results to the list
        results.append({
            "ACK": predictions["ACK"],
            "BCC": predictions["BCC"],
            "MEL": predictions["MEL"],
            "NEV": predictions["NEV"],
            "SCC": predictions["SCC"],
            "SEK": predictions["SEK"],
            "Prediction": predicted_class_name,
            "Real_class": real_class
        })

    # Create a data frame from the results list
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
def multi_class_accuracy_score(model):
    """
    This function calculates the multi-class accuracy score.
    """
    predictions_df = predict_for_all_customers(model)

    # Get unique classes for which predictions were made
    unique_classes = predictions_df['Real_class'].unique()

    # Filter the DataFrame to include only rows with the unique classes
    grouped_df = predictions_df[predictions_df['Real_class'].isin(unique_classes)]

    # Group by 'Real_class' and sum the predictions for each class
    grouped_df = grouped_df.groupby('Real_class').sum()

    # Sort the columns to ensure the same order
    grouped_df = grouped_df.reindex(sorted(grouped_df.columns), axis=1)

    # Convert the DataFrame to a numeric data type (convert any non-numeric values to NaN)
    grouped_df = grouped_df.apply(pd.to_numeric, errors='coerce')

    # Fill NaN values with zeros
    grouped_df.fillna(0, inplace=True)

    # Calculate the accuracy score
    accuracy = np.trace(grouped_df) / grouped_df.to_numpy().sum()
    accuracy = round(accuracy, 6)

    return accuracy


In [ ]:
accuracy_score = multi_class_accuracy_score(model)

In [ ]:
accuracy_score